In [3]:
import time
import json
import requests
from collections import defaultdict
import pandas as pd
from dbConn.get_query import get_select, get_insert

In [2]:
query = 'select pk, latlng from scrapping.scrapping.instagram_location;'
locs = get_select(query)

In [3]:
len(locs)

74864

In [5]:
url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=129.12763322949&y=35.172803805612"
headers = {"Authorization": "KakaoAK 35411bfa614a5dd447f4c6b3a3b95559"}
api_test = requests.get(url, headers=headers)
url_text = json.loads(api_test.text)
url_text

{'meta': {'total_count': 2},
 'documents': [{'region_type': 'B',
   'code': '2635010500',
   'address_name': '부산광역시 해운대구 우동',
   'region_1depth_name': '부산광역시',
   'region_2depth_name': '해운대구',
   'region_3depth_name': '우동',
   'region_4depth_name': '',
   'x': 129.14378876810525,
   'y': 35.172340282874565},
  {'region_type': 'H',
   'code': '2635052000',
   'address_name': '부산광역시 해운대구 우2동',
   'region_1depth_name': '부산광역시',
   'region_2depth_name': '해운대구',
   'region_3depth_name': '우2동',
   'region_4depth_name': '',
   'x': 129.13999879270276,
   'y': 35.17127909456614}]}

In [6]:
headers = {"Authorization": "KakaoAK 35411bfa614a5dd447f4c6b3a3b95559"}
api_url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?"
latlng_address = defaultdict(list)

for (pk, latlng,) in locs:
    if latlng:
        y, x = eval(latlng)
        url = api_url + f'x={x}&y={y}'
        try:
            api = requests.get(url, headers=headers)
        except:
            print(pk)
            break
        api_text = json.loads(api.text)

        if 'documents' in api_text.keys():
            result = api_text['documents'][1]
            latlng_address[pk] = result['address_name']

In [10]:
import pickle

with open('./latlng_address.json', 'wb') as f:
    pickle.dump(latlng_address, f)

In [7]:
with open('./latlng_addres.pkl', 'rb') as f:
    latlng_address = pickle.load(f)
len(latlng_address)

66590

In [4]:
import pickle

with open('./valid_location_pk2.json', 'rb') as f:
    latlng_address2 = pickle.load(f)

In [8]:
for pk, addr in latlng_address2.items():
    latlng_address[pk] = addr
len(latlng_address)

71623

In [ ]:
for pk, addr in latlng_address.items():
    query = f"update scrapping.scrapping.instagram_location set address = '{addr}' where pk = {pk};"
    get_insert(query)